In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/ykotseruba/JAAD.git
!cd JAAD && bash download_clips.sh

In [24]:
import cv2
import os
def extractresize(video_path, video_file, output_folder):
  video_name = os.path.splitext(video_file)[0]
  # Membaca video
  cap = cv2.VideoCapture(os.path.join(video_path,video_file))
  # Menghitung frame
  count = 0
  # Iterasi selama video terbaca
  while cap.isOpened():
    # Mengambil frame dalam video
    success, frame = cap.read()
    # Proses berlangsung jika frame berhasil dibaca
    if success:
      # Ubah ukuran frame
      resize = cv2.resize(frame, (640,640))
      # Write Image with 90% quality
      cv2.imwrite(f'{output_folder}/{video_name}/{count:05}.jpg',resize, [cv2.IMWRITE_JPEG_QUALITY, 80])
      count+=1
    else:
      cap.release()

In [4]:
!cd JAAD && mkdir images

In [23]:
import os
%cd JAAD/images
for i in [f"video_{x:04}" for x in range(1,347)]:
  os.mkdir(i)

/content/JAAD/images


In [14]:
video_path = '/content/JAAD/JAAD_clips'
video_files = os.listdir(video_path)
video_files.sort()

In [25]:
count = 0
for video_file in video_files:
  count+=1
  print(count)
  extractresize(video_path, video_file, '/content/JAAD/images')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
data_opts = {'fstride': 1,
            'sample_type': 'all',
            'subset': 'all_videos',
            'data_split_type': 'default',
            'seq_type': 'trajectory',
            'height_rng': [0, float('inf')],
            'squarify_ratio': 0,
            'min_track_size': 0,
            'random_params': {'ratios': None,
                              'val_data': True,
                              'regen_data': True},
            'kfold_params': {'num_folds': 5, 'fold': 1}}

In [4]:
%cd JAAD
from jaad_data import JAAD
jaad_path = '/content/JAAD'
imdb = JAAD(data_path=jaad_path)

/content/JAAD


In [7]:
!rm -r /content/yolodata

In [11]:
imdb.get_detection_data('train', 'yolo3', occlusion_type='full', file_path='/content/', **data_opts)

---------------------------------------------------------
Generating database for jaad
jaad database loaded from /content/JAAD/data_cache/jaad_database.pkl
Number of unique pedestrians 1941 
Number of samples 284043 
Data generated for YOLO3


('/content/yolo3_train.txt', '/content/mapping_yolo3')

In [12]:
imdb.get_detection_data('val', 'yolo3', occlusion_type='full', file_path='/content/', **data_opts)

---------------------------------------------------------
Generating database for jaad
jaad database loaded from /content/JAAD/data_cache/jaad_database.pkl
Number of unique pedestrians 762 
Number of samples 94600 
Data generated for YOLO3


('/content/yolo3_val.txt', '/content/mapping_yolo3')

In [17]:
!mkdir yolodata
!cd yolodata && mkdir train val
!cd yolodata/train && mkdir images labels
!cd yolodata/val && mkdir images labels

In [18]:
import os
import shutil
def yolov3toyolov8(labelfile,output_dir,split):
  with open(labelfile, 'r') as f:
    lines = [line.rstrip() for line in f]
  for line in lines:
    imgpath, *boxes = line.split()
    video_name = imgpath.split('/')[-2]
    frame_id = os.path.splitext(imgpath.split('/')[-1])[0]
    shutil.copy(imgpath,f"/content/yolodata/{split}/images/{video_name}_{frame_id}.jpg")
    output_file = f'{video_name}_{frame_id}.txt'
    with open(os.path.join(output_dir,output_file), 'w') as f:
      for box in boxes:
        box = [int(num) for num in box.split(',')]
        x0, y0, x1, y1, cls = box
        x = (x1+x0)/(2*1920)
        y = (y1+y0)/(2*1080)
        w = (x1-x0)/1920
        h = (y1-y0)/1080
        f.write(f"{cls} {x} {y} {w} {h}\n")

In [26]:
yolov3toyolov8('/content/yolo3_train.txt','/content/yolodata/train/labels','train')

In [27]:
yolov3toyolov8('/content/yolo3_val.txt','/content/yolodata/val/labels','val')

In [28]:
%cd /content/yolodata
!zip -r yolodata90.zip train val

Streaming output truncated to the last 5000 lines.
  adding: val/images/video_0270_00025.jpg (deflated 0%)
  adding: val/images/video_0311_00113.jpg (deflated 1%)
  adding: val/images/video_0277_00328.jpg (deflated 1%)
  adding: val/images/video_0256_00082.jpg (deflated 2%)
  adding: val/images/video_0268_00008.jpg (deflated 0%)
  adding: val/images/video_0325_00091.jpg (deflated 1%)
  adding: val/images/video_0311_00021.jpg (deflated 1%)
  adding: val/images/video_0332_00012.jpg (deflated 0%)
  adding: val/images/video_0291_00152.jpg (deflated 1%)
  adding: val/images/video_0269_00204.jpg (deflated 1%)
  adding: val/images/video_0334_00093.jpg (deflated 0%)
  adding: val/images/video_0266_00335.jpg (deflated 0%)
  adding: val/images/video_0254_00095.jpg (deflated 0%)
  adding: val/images/video_0271_00246.jpg (deflated 0%)
  adding: val/images/video_0322_00241.jpg (deflated 0%)
  adding: val/images/video_0262_00122.jpg (deflated 0%)
  adding: val/images/video_0271_00206.jpg (deflated 0

In [29]:
!cp yolodata90.zip /content/gdrive/MyDrive